In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#tf.enable_eager_execution(config=config)
import os
from time import time

/home/james/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
keras = tf.keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

Using TensorFlow backend.


In [3]:
# Hyperparameters
cores = os.cpu_count()
batch_size = 128
num_classes = 10
epochs = 10
buffer_size = 100000

# Load mnist Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_height, img_width = x_train.shape[1],x_train.shape[2]

# reshape image to 4D Tensor
x_train = x_train.reshape([-1,img_height,img_width,1]).astype("float32")
x_test = x_test.reshape([-1,img_height,img_width,1]).astype("float32")

# convert to one hot encoing 
y_train = tf.one_hot(y_train, num_classes, dtype=tf.float32)
y_test = tf.one_hot(y_test, num_classes, dtype=tf.float32)

In [4]:
layers = tf.keras.layers

model = tf.keras.Sequential()
model.add(layers.Conv2D(64,3, padding="same", activation="relu", input_shape=(img_height,img_width,1)))
model.add(layers.Conv2D(64,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(256,3, padding="same", activation="relu"))
model.add(layers.Conv2D(256,3, padding="same", activation="relu"))
model.add(layers.Conv2D(256,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(512,3, padding="same", activation="relu"))
model.add(layers.Conv2D(512,3, padding="same", activation="relu"))
model.add(layers.Conv2D(512,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dense(num_classes, activation="softmax"))

# print the model summary
#model.summary()

optimizer = tf.train.AdamOptimizer()
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [6]:
estimator_model = keras.estimator.model_to_estimator(keras_model=model,
                                                    model_dir="./tf_model")

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_session_config': None, '_save_checkpoints_steps': None, '_service': None, '_model_dir': './tf_model', '_is_chief': True, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff78c3ebf28>, '_train_distribute': None, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_tf_random_seed': None, '_evaluation_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000}


In [7]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(buffer_size=buffer_size)
dataset = dataset.batch(batch_size=batch_size)
dataset = dataset.repeat(epochs)
dataset = dataset.prefetch(2)

In [9]:
before = time()
with tf.Session() as sess:
    dataset = sess.run(dataset)
estimator_model.train(dataset)
after = time()

print("Total time to train : ", after-before)

TypeError: Fetch argument <PrefetchDataset shapes: ((?, 28, 28, 1), (?, 10)), types: (tf.float32, tf.float32)> has invalid type <class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>, must be a string or Tensor. (Can not convert a PrefetchDataset into a Tensor or Operation.)